Evaluate the effects of using the full dataset on the HMM and DistilBERT, compare results and effectiveness of using both including training times etc.

# HMM Test

In [1]:
# from helper import accuracy, entity_count
%pip install datasets
import nltk
from sklearn.metrics import precision_recall_fscore_support
from datasets import load_dataset, load_metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, load_metric
dataset = load_dataset("surrey-nlp/PLOD-Filtered")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [4]:
trainSentences = dataset["train"][:]["tokens"]
trainLabels = dataset["train"][:]["ner_tags"]

print(len(trainSentences))

print(trainSentences)
print(trainLabels)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# We need the unique set of tokens
# We need the unique set of labels (pre-post conversion?)

symbolSet = set()
for tokenSet in trainSentences:
  for token in tokenSet:
    for char in token:
      symbolSet.add(char)

# Get the unique set of labels
entitySet = set()
for tagSet in trainLabels:
  for tag in tagSet:
    entitySet.add(tag)

In [6]:
entitySet = list(entitySet) # Convert back to lists
print(entitySet)

symbolSet = list(symbolSet)
print(symbolSet)
print(len(symbolSet))

[0, 1, 3, 4]
['`', 'к', '™', '„', 'Ф', 'p', '‘', 'Χ', 't', '⑧', 'Á', ']', 'â', '!', '«', 'Φ', 'u', '⑯', '⑱', '∇', 'қ', 'Β', '[', '·', 'á', '➔', '⇔', '"', 'ξ', 'b', '∝', 'T', '≙', '）', '¶', '0', 'W', 'Z', '5', 'e', 'q', 'J', '☆', '6', 'x', 'Å', '⌋', '≅', 'ǁ', 'ӧ', '▄', 'P', 'É', 'd', 'μ', '⑤', '÷', 'H', '>', 'ι', 'S', 'R', 'O', 'Κ', 'Γ', '̂', 'k', '�', 'Y', '€', '»', '*', '♂', '΄', '◇', 'ϖ', '⑮', '∶', '≥', '…', 'Ѱ', 'η', '≧', 'Ā', 'ƞ', '˙', 'Ο', '∩', '≔', ':', 'Ś', '▼', '‐', 'õ', 'h', '˜', '↓', 'β', ',', '∅', 'κ', 'ô', 'ν', 'ɸ', '—', 'ú', 'М', 'º', '︸', '、', '¼', 'Δ', '⁄', '¤', 'ö', 'Ɛ', 'ƛ', '±', 'F', '%', '#', 'Ψ', '°', '½', '◻', 'ϵ', '∼', 'm', 'A', 'Ω', '≠', '\\', 'ï', '⌊', 'υ', '¯', 'σ', '※', 'y', '⑥', '⑪', '+', '○', 'ˊ', '?', 'ß', 'ε', '□', 'ý', 'U', 'ɛ', 'C', '↑', '≦', '×', 'L', '⊥', 'µ', 'N', 'ᵒ', '⟞', '⑭', 'ę', '〖', '⩾', 'X', '–', 'τ', '∀', 'V', 'g', '(', 'B', 'D', '⑬', '℃', '↔', '8', '◊', 'ê', 'ϒ', '7', '⇒', '⟩', '●', 'ó', 'ş', '★', '∆', '‑', 'j', 'Ѓ', 'ϴ', 'ψ', 'ʎ', '√', 'I', 

In [7]:
# Flatten the training data for easy use later
flat_inputs_train = [item for sublist in train_dataset["tokens"] for item in sublist]
flat_labels_train = [item for sublist in train_dataset["ner_tags"] for item in sublist]

In [8]:
trainCombinedOld = [[(token, label) for token, label in zip(tokenSet, labelSet)] for tokenSet, labelSet in zip(trainSentences, trainLabels)]

trainCombined = []
for tokens, labels in zip(trainSentences, trainLabels):
    sequence = [(token, label) for token, label in zip(tokens, labels)]
    trainCombined.append(sequence)

print(trainCombined[0])
print(trainCombinedOld[0])

[('Alternatively', 0), (',', 0), ('fibroblasts', 0), ('were', 0), ('plated', 0), ('sparsely', 0), ('so', 0), ('that', 0), ('they', 0), ('did', 0), ('not', 0), ('touch', 0), ('each', 0), ('other', 0), ('and', 0), ('induced', 0), ('into', 0), ('quiescence', 0), ('by', 0), ('serum', 0), ('starvation', 0), ('and', 0), ('monitored', 0), ('after', 0), ('4', 0), ('d', 0), ('(', 0), ('serum', 3), ('-', 4), ('starved', 4), ('for', 4), ('4', 4), ('d', 4), ('[', 0), ('SS4', 1), (']', 0), (')', 0), ('or', 0), ('7', 0), ('d', 0), ('(', 0), ('serum', 3), ('-', 4), ('starved', 4), ('for', 4), ('7', 4), ('d', 4), ('[', 0), ('SS7', 1), (']', 0), (')', 0), ('.', 0)]
[('Alternatively', 0), (',', 0), ('fibroblasts', 0), ('were', 0), ('plated', 0), ('sparsely', 0), ('so', 0), ('that', 0), ('they', 0), ('did', 0), ('not', 0), ('touch', 0), ('each', 0), ('other', 0), ('and', 0), ('induced', 0), ('into', 0), ('quiescence', 0), ('by', 0), ('serum', 0), ('starvation', 0), ('and', 0), ('monitored', 0), ('after',

In [9]:
trainerSetup = nltk.tag.hmm.HiddenMarkovModelTrainer(states = entitySet, symbols = symbolSet)

model = trainerSetup.train_supervised(trainCombined)

In [10]:
# valSentences = val_dataset["tokens"]
# valLabels = val_dataset["ner_tags"]
import numpy as np

flatValSentences = [item for sublist in val_dataset["tokens"] for item in sublist]
flatValLabels = [item for sublist in val_dataset["ner_tags"] for item in sublist]

test_sentences = dataset["test"][:]["tokens"]

# Predict with HMM, you need to tag each token individually rather than pass in the entire list otherwise it will predict everything as one item!
# preds = []
# for token in flatValSentences:
#   preds.append(model.tag(token))

predicted = []
for sentence in test_sentences:
    test_result = model.tag(sentence)
    out_tags = []
    for word, tag in test_result:
        out_tags.append(tag)
    predicted.append(out_tags)
predicted[0]

/usr/local/lib/python3.10/dist-packages/nltk/tag/hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
/usr/local/lib/python3.10/dist-packages/nltk/tag/hmm.py:364: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [11]:
predTags = [tag for tagList in predicted for tag in tagList]

print(set(predTags))

print(len(predTags))

{0, 1, 3, 4}
1129666


In [ ]:
print(flatValLabels)

In [13]:
flatTestLabels = [item for sublist in test_dataset["ner_tags"] for item in sublist]

In [14]:
print(len(flatTestLabels))

1129666


In [15]:
from sklearn.metrics import classification_report

print(classification_report(flatTestLabels, predTags))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94    937431
           1       0.87      0.68      0.77     66240
           3       0.63      0.52      0.57     39288
           4       0.68      0.59      0.63     86707

    accuracy                           0.90   1129666
   macro avg       0.78      0.69      0.73   1129666
weighted avg       0.89      0.90      0.89   1129666

